# Interacting with the API as a user with OAuth

Before running this notebook, log in to LifeMonitor and register an OAuth app. This will be your LifeMonitor client.

* Choose a Client Name (e.g., MyLMClient)
* Enter arbitrary URIs for "Client URI" and "Client Redirect URIs"
* Enable all scopes in "Allowed scopes"
* Set "Confidential" to "On" in Client Type and set "Client Authentication Method" to `client_secret_post`

In [1]:
import requests
import uuid
from urllib.parse import urlparse
from requests.auth import HTTPBasicAuth

In [2]:
lm_base_url = "https://api.dev.lifemonitor.eu"
lm_token_url = f"{lm_base_url}/oauth2/token"
# Enter the redirect URI from the OAuth app configuration
redirect_uri = f"https://example.com"

In [3]:
# Get these from your LifeMonitor OAuth app
CLIENT_ID = "???"
CLIENT_SECRET = "???"

In [4]:
# Your LifeMonitor username and password
USERNAME = "SimoneLeo"
PASSWORD = "???"

In [5]:
s = requests.session()
s.verify = False
auth = HTTPBasicAuth(USERNAME, PASSWORD)

In [6]:
# authorization request
params = {
    "client_id": CLIENT_ID,
    "response_type": "code",
    "state": uuid.uuid4().hex,
    "confirm": False,
    "redirect_uri": redirect_uri,
    "scope": "registry.info user.profile user.workflow.read user.workflow.write workflow.read workflow.write testingService.read testingService.write",
}
auth_response = s.get(f"{lm_base_url}/oauth2/authorize", params=params, auth=auth, verify=False)
assert auth_response.status_code == 200

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [7]:
auth_response.url

'https://api-lm-test.rahtiapp.fi/static/oauth-receiver.html?code=FrQVZ4AHBmrVSXCgpl84A7TSbsbVOJLgAV73zS1GiRDv6ty3&state=f3586cd732d74d59be9d077cc3794cbe'

In [8]:
# get the authorization code
location = urlparse(auth_response.url)
query_params = location.query.split('&')
code = query_params[0].replace("code=", "")
code

'FrQVZ4AHBmrVSXCgpl84A7TSbsbVOJLgAV73zS1GiRDv6ty3'

In [9]:
# Get an authorization token from LifeMonitor
token_response = s.post(
    lm_token_url, 
    data={
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "authorization_code",
        "redirect_uri": redirect_uri,
        "code": code,
    })
token_response.content
assert token_response.status_code == 200
token = token_response.json()
token

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'access_token': 'f6GTi5ST4nKaajxhAqPyULoIpahBJDmCeLi4NOJpWF',
 'expires_in': 864000,
 'scope': 'registry.info user.profile user.workflow.read user.workflow.write workflow.read workflow.write testingService.read testingService.write',
 'token_type': 'Bearer'}

In [10]:
# Update headers with the OAuth2 token
s.headers.update({'Authorization': f"Bearer {token['access_token']}"})

In [11]:
# List workflows
response = s.get(f"{lm_base_url}/workflows", params={"status": False})
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_workflows = response.json()
registry_workflows

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'items': [{'latest_version': '1',
   'name': 'sort-and-change-case-travis',
   'uuid': 'bc435030-6e1e-0139-2dea-005056ab5db4'}],
 'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://api-lifemonitor-test.rahtiapp.fi',
  'resource': '/workflows?status=False'}}